<a href="https://colab.research.google.com/github/nohwiin/ML/blob/master/Capstone2020/generate_img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
import os
import random
import numpy as np
import errno
import csv
from PIL import Image
import matplotlib.pyplot as plt
import cv2

In [ ]:
import numpy

def find_coeffs(pa, pb):
    matrix = []
    for p1, p2 in zip(pa, pb):
        matrix.append([p1[0], p1[1], 1, 0, 0, 0, -p2[0]*p1[0], -p2[0]*p1[1]])
        matrix.append([0, 0, 0, p1[0], p1[1], 1, -p2[1]*p1[0], -p2[1]*p1[1]])

    A = numpy.matrix(matrix, dtype=numpy.float)
    B = numpy.array(pb).reshape(8)

    res = numpy.dot(numpy.linalg.inv(A.T * A) * A.T, B)
    return numpy.array(res).reshape(8)

In [ ]:
def compose_images(foreground_path, background_path):
    foreground = Image.open(foreground_path)

    background = Image.open(background_path)
    background = background.convert('RGBA')
    background = background.resize((512,512))

    # Rotate the foreground
    angle_degrees = random.randint(0, 359)
    foreground = foreground.rotate(angle_degrees, resample=Image.BICUBIC, expand=True)

    # Pick something between .5 and 1
    scale = random.uniform(0.5, 1)
    new_size = (int(512 * scale), int(512 * scale))
    foreground = foreground.resize(new_size, resample=Image.BICUBIC)

    # Choose a random x,y position for the foreground
    max_xy_position = (background.size[0] - foreground.size[0], background.size[1] - foreground.size[1])
    paste_position = (random.randint(0, max_xy_position[0]), random.randint(0, max_xy_position[1]))

    # Create a new foreground image as large as the background and paste it on top
    new_foreground = Image.new('RGBA', background.size, color = (0, 0, 0, 0))
    new_foreground.paste(foreground, paste_position)
        
    # Extract the alpha channel from the foreground and paste it into a new image the size of the background
    alpha_mask = foreground.getchannel(3)
    new_alpha_mask = Image.new('L', background.size, color=0)
    new_alpha_mask.paste(alpha_mask, paste_position)
    composite = Image.composite(new_foreground, background, new_alpha_mask)
    
    # Grab the alpha pixels above a specified threshold
    alpha_threshold = 200
    mask_arr = np.array(np.greater(np.array(new_alpha_mask), alpha_threshold), dtype=np.uint8)
    hard_mask = Image.fromarray(np.uint8(mask_arr) * 255, 'L')

    # Get the smallest & largest non-zero values in each dimension and calculate the bounding box
    nz = np.nonzero(hard_mask)
    padding_percent = 0.1

    bbox_w = np.max(nz[1]) - np.min(nz[1])
    bbox_w_padding = int(bbox_w * padding_percent)
    bbox_h = np.max(nz[0]) - np.min(nz[0])
    bbox_h_padding = int(bbox_h * padding_percent)

    left = np.min(nz[1]) - bbox_w_padding
    if left < 0:
      left = 0
    right = np.max(nz[1]) + bbox_w_padding
    if right > 511:
      right = 511
    top = np.min(nz[0]) - bbox_h_padding
    if top < 0:
      top = 0
    bottom = np.max(nz[0]) + bbox_h_padding
    if bottom > 511:
      bottom = 511

    bbox = [left, top, right, bottom]

    bbox_tuple = tuple(bbox)
    composite = composite.crop(bbox_tuple)
    composite = composite.resize((512,384))

    return composite, hard_mask, bbox

In [ ]:
# Get lists of foreground and background image paths
dataset_dir = '/gdrive/My Drive/Capstone2020'
#Label = ['Can','Glass','Vinyl','PET','Plastic','Paper','Paperpack']
Label = ['Paperpack']

backgrounds_dir = os.path.join(dataset_dir, 'background')
foregrounds_dir = os.path.join(dataset_dir, 'foreground')
backgrounds = [os.path.join(backgrounds_dir, file_name) for file_name in os.listdir(backgrounds_dir)]

output_dir = '/gdrive/My Drive/Capstone2020/Dataset/fake'

try:
  os.mkdir(output_dir)
except OSError as exc:
  if exc.errno != errno.EEXIST:
    raise
  pass

for lb in Label:
  save_dir = os.path.join(output_dir, lb)
  try:
    os.mkdir(save_dir)
  except OSError as exc:
    if exc.errno != errno.EEXIST:
      raise
    pass
  label_dir = os.path.join(foregrounds_dir, lb)
  foregrounds = [os.path.join(label_dir, file_name) for file_name in os.listdir(label_dir)]

  foreground_idx = 0
  iter_num = 760
  for itr in range(iter_num):
    background = random.choice(backgrounds)
    composite, mask, bbox = compose_images(foregrounds[foreground_idx], background)
    composite_path = os.path.join(save_dir, lb+'{0:04d}.png'.format(itr+1))
    composite.save(composite_path)
    foreground_idx+=1
    if foreground_idx == len(foregrounds):
      foreground_idx = 0